In [1]:
#Required Modules to build our model
from __future__ import print_function
from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.


In [2]:
#Model Parameters
maxlen = 30
labels = 2

In [3]:
#Importing the data
input = pd.read_csv("gender_data.csv",header=None)

In [4]:
#Printing the data that is imported
input

,0,1
0,barjraj,m
1,ramdin verma,m
2,sharat chandran,m
3,birender mandal,m
4,amit,m
5,kushal,m
6,kasid,m
7,shiv prakash,m
8,vikram singh,m
9,sanjay,m


In [5]:
#Setting the coloumn names
input.columns = ['name','m_or_f']

In [6]:
input.columns 

Index([u'name', u'm_or_f'], dtype='object')

In [7]:
input

,name,m_or_f
0,barjraj,m
1,ramdin verma,m
2,sharat chandran,m
3,birender mandal,m
4,amit,m
5,kushal,m
6,kasid,m
7,shiv prakash,m
8,vikram singh,m
9,sanjay,m


In [8]:
#Calculating the length of Each Name and adding the length coloun to the input
input['lengthofthename']= [len(str(i)) for i in input['name']]

In [9]:
input['lengthofthename']

0         7
1        12
2        15
3        15
4         4
5         6
6         5
7        12
8        12
9         6
10        4
11       14
12       12
13       12
14       10
15        4
16        7
17        6
18        7
19       11
20        9
21       11
22       15
23        5
24        6
25        7
26       16
27        6
28        7
29       18
         ..
15260    12
15261     6
15262    11
15263    11
15264    18
15265     8
15266    15
15267    23
15268     7
15269    13
15270    13
15271    16
15272    11
15273     5
15274     9
15275     5
15276    12
15277     6
15278     5
15279    11
15280    12
15281    13
15282    11
15283     8
15284    10
15285     1
15286    10
15287    11
15288    23
15289     5
Name: lengthofthename, Length: 15290, dtype: int64

In [10]:
#Consider the names whose length is >1 and creating a dataset
input1 = input[(input['lengthofthename'] >= 2) ]

In [11]:
input1

,name,m_or_f,lengthofthename
0,barjraj,m,7
1,ramdin verma,m,12
2,sharat chandran,m,15
3,birender mandal,m,15
4,amit,m,4
5,kushal,m,6
6,kasid,m,5
7,shiv prakash,m,12
8,vikram singh,m,12
9,sanjay,m,6


In [12]:
#Calculating the count of each type
input1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [13]:
names = input['name']

In [14]:
names

0                        barjraj
1                   ramdin verma
2                sharat chandran
3                birender mandal
4                           amit
5                         kushal
6                          kasid
7                   shiv prakash
8                   vikram singh
9                         sanjay
10                          abhi
11                ram dutt gupta
12                  khadak singh
13                  gurmit singh
14                    chanderpal
15                          aman
16                       khursid
17                        rajeev
18                       durgesh
19                   nahar singh
20                     ram kumar
21                   sunder paal
22               maansingh aswal
23                         rohit
24                        sparsh
25                       santosh
26              punit khandelwal
27                        dinesh
28                       gulshan
29            arvind kumar yadav
          

In [15]:
gender = input['m_or_f']

In [16]:
gender

0        m
1        m
2        m
3        m
4        m
5        m
6        m
7        m
8        m
9        m
10       m
11       m
12       m
13       m
14       m
15       m
16       m
17       m
18       m
19       m
20       m
21       m
22       m
23       m
24       m
25       m
26       m
27       m
28       m
29       m
        ..
15260    f
15261    f
15262    f
15263    f
15264    f
15265    f
15266    f
15267    f
15268    f
15269    f
15270    f
15271    f
15272    f
15273    f
15274    f
15275    f
15276    f
15277    f
15278    f
15279    f
15280    f
15281    f
15282    f
15283    f
15284    f
15285    f
15286    f
15287    f
15288    f
15289    f
Name: m_or_f, Length: 15290, dtype: object

In [17]:
#Building the Vocabulary
vocab = set(' '.join([str(i) for i in names]))

In [18]:
vocab.add('END')

In [19]:
vocab.add('END')
len_vocab = len(vocab)
print(vocab)
print("vocabulary length is ",len_vocab)
print ("length of input is ",len(input1))

set([' ', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'])
vocabulary length is  39
length of input is  15226


In [20]:
#Building a dictionary for the vocabulary
char_index = dict((c, i) for i, c in enumerate(vocab))
print(char_index)

{' ': 0, '.': 1, '1': 2, '0': 3, '3': 4, '2': 5, '5': 6, '4': 7, '7': 8, '6': 9, '9': 10, '8': 11, 'END': 12, 'a': 13, 'c': 14, 'b': 15, 'e': 16, 'd': 17, 'g': 18, 'f': 19, 'i': 20, 'h': 21, 'k': 22, 'j': 23, 'm': 24, 'l': 25, 'o': 26, 'n': 27, 'q': 28, 'p': 29, 's': 30, 'r': 31, 'u': 32, 't': 33, 'w': 34, 'v': 35, 'y': 36, 'x': 37, 'z': 38}


In [21]:
#Dividing the data set into testing and training
msk = np.random.rand(len(input1)) < 0.8
train = input1[msk]
test = input1[~msk]     

In [22]:
train

,name,m_or_f,lengthofthename
0,barjraj,m,7
1,ramdin verma,m,12
3,birender mandal,m,15
4,amit,m,4
5,kushal,m,6
6,kasid,m,5
7,shiv prakash,m,12
8,vikram singh,m,12
9,sanjay,m,6
10,abhi,m,4


In [23]:
#The results are random
test

,name,m_or_f,lengthofthename
2,sharat chandran,m,15
11,ram dutt gupta,m,14
13,gurmit singh,m,12
15,aman,m,4
18,durgesh,m,7
24,sparsh,m,6
27,dinesh,m,6
36,raj kumar,m,9
38,sevak pitambar lal,m,18
42,dharam singh,m,12


In [24]:
len(test)

3116

In [25]:
len(train)

12110

In [26]:
float(len(test))/len(train)

0.25730800990916597

In [27]:
#Basic type of encoding called the one hot encding It only activates a one which represents the character and remaining are zeros
def encoding(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [28]:
encoding(4)

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [29]:
encoding(37)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.])

In [30]:
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [encoding(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(encoding(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])

In [31]:
np.asarray(train_X).shape

(12110, 30, 39)

In [32]:
np.asarray(train_Y).shape

(12110, 2)

In [33]:
print('Build Network...')
#Sequential is  a basic type of Data structure in Keras Module it is like a container
Network = Sequential()


Build Network...


In [34]:
#
Network.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
Network.add(Dropout(0.2))
Network.add(LSTM(512, return_sequences=False))
Network.add(Dropout(0.2))
Network.add(Dense(2))
Network.add(Activation('softmax'))
Network.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [35]:
#Creating the test dataset 
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [encoding(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(encoding(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])

In [36]:
(np.asarray(test_X).shape)

(3116, 30, 39)

In [37]:
np.asarray(test_Y).shape

(3116, 2)

In [38]:
Network.fit(train_X, train_Y,batch_size=1000,epochs=30,validation_data=(test_X, test_Y))

Train on 12110 samples, validate on 3116 samples
Epoch 1/30
12110/12110 [==============================] - 177s - loss: 0.6887 - acc: 0.5618 - val_loss: 0.6863 - val_acc: 0.5485
Epoch 2/30
12110/12110 [==============================] - 170s - loss: 0.6652 - acc: 0.5943 - val_loss: 0.6433 - val_acc: 0.6852
Epoch 3/30
12110/12110 [==============================] - 179s - loss: 0.6148 - acc: 0.6703 - val_loss: 0.5947 - val_acc: 0.6823
Epoch 4/30
12110/12110 [==============================] - 173s - loss: 0.5817 - acc: 0.6964 - val_loss: 0.5641 - val_acc: 0.7160
Epoch 5/30
12110/12110 [==============================] - 283s - loss: 0.5418 - acc: 0.7303 - val_loss: 0.5269 - val_acc: 0.7333
Epoch 6/30
12110/12110 [==============================] - 316s - loss: 0.5129 - acc: 0.7509 - val_loss: 0.5126 - val_acc: 0.7465
Epoch 7/30
12110/12110 [==============================] - 256s - loss: 0.4880 - acc: 0.7668 - val_loss: 0.5009 - val_acc: 0.7664
Epoch 8/30
12110/12110 [========================

In [62]:
#Evaluating on Test data
score, acc = Network.evaluate(test_X, test_Y)

3116/3116 [==============================] - 21s    


In [40]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.330644838847
Test accuracy: 0.868741976893


In [48]:
#Test the Network with the names that are not in the dataset
name=["pradeep","ravi","srinu","udaya","lakshmi","siva","kumari","pamulaiah","vivek","ramarao","krish","mrs. rajini","pretham","sravan","ragini rajaram"]

In [50]:
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [encoding(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(encoding(char_index["END"]))
    X.append(tmp)

In [60]:
prediction=Network.predict(np.asarray(X))

In [63]:
prediction

array([[ 0.97539312,  0.02460693],
       [ 0.18862501,  0.81137496],
       [ 0.22428751,  0.77571255],
       [ 0.24406148,  0.75593853],
       [ 0.22737789,  0.77262211],
       [ 0.18423903,  0.81576097],
       [ 0.0693362 ,  0.93066382],
       [ 0.07213493,  0.92786503],
       [ 0.96454453,  0.03545545],
       [ 0.07393117,  0.9260689 ],
       [ 0.68024886,  0.31975111],
       [ 0.03066618,  0.96933383],
       [ 0.80558592,  0.19441403],
       [ 0.52798164,  0.47201839],
       [ 0.35345128,  0.64654863]], dtype=float32)

In [64]:
#Saves the Calculated Weights
Network.save_weights('gender_Network',overwrite=True)